# Motivation
论文原文：[《Product-based Neural Networks for User Response Prediction》](https://arxiv.org/pdf/1611.00144)

参考学习地址：[详解 PNN: 在NN中显式引入Product](https://zhuanlan.zhihu.com/p/56651241)

相比Embedding+MLP的传统结构（NN层之间用add operation，通过激活函数引入非线性），PNN认为“add”也许不足以捕获不同的field特征间的相关性（也即FNN的思想），因为FM的本质思想是内“积”（也即乘法），所以在embedding层后设计了Product Layer，以显示捕捉基于Field的二阶特征相关性。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/pnn.png" width = "500" height = "450" alt="PNN" align=center />

## Product Layer
L1层输入包含两个部分：
* 左边Z部分，通过和1相乘，其实就是将Embedding层学到的嵌入表征原封不动地搬来
* 右边P部分，每个节点是两两Field的embedding对应的product结果。根据Product函数的不同选择，PNN也有不同实现，文中尝试了相对常见的向量内积和向量外积（见Wide&Deep中向量操作），分别对应了IPNN和OPNN。

$$
\begin{align}
l_1=relu(l_z+l_p+b_1)\in \R^D
\end{align}
$$

虽然结构简单，但在工业场景中，product层的节点数至少是输入field数的平方级别，而实际上field的规模往往已经很大了，PNN落地构成了挑战。

### IPNN
IPNN（向量内积），从矩阵视角看，等价于：

$$
\begin{align}
\mathbf{P}\in \R^{N*N} \text{, where } P_{ij}=<f_i,f_j>=f_i^Tf_j\in \R
\end{align}
$$

其中 N表示输入的field个数，$f_i \in \R^M$ 表示第i个field的embedding向量。$l_p=(l_p^1,l_p^2,...,l_p^d,...,l_p^D)$

$$
\begin{align}
l_p^d=\mathbf{W}_p^d\odot \mathbf{P} = \Sigma_{i=1}^N\Sigma_{j=1}^N(\mathbf{W}_p^d)_{ij}\mathbf{P}_{ij}
\end{align}
$$

其中，$\mathbf{W}_p^d\in \R^{N*N}$表示第d个节点连接product层的权重，$\odot$代表哈达玛积（见Wide&Deep中向量操作）。注意到$\mathbf{P}$和$\mathbf{W}_p^d$都是对称矩阵，可以类似FM进行矩阵分解，另$\mathbf{W}_p^d=\theta^d(\theta^d)^T, \theta^d\in \R^N$，则

$$
\begin{align}
l_p^d=\mathbf{W}_p^d\odot \mathbf{P} = \Sigma_{i=1}^N\Sigma_{j=1}^N\theta_i^d(\theta_j^d)^T<f_i,f_j>=<\Sigma_{i=1}^N\delta_i^d,\Sigma_{i=1}^N\delta_i^d>=||\Sigma_{i=1}^N\delta_i^d||^2 \text{, where } \delta_i^d=\theta_i^df_i
\end{align}
$$

由此，L1层的空间复杂度$O(DNM+DN^2)\rightarrow O(DNM)$，时间复杂度$O(DNM+DN^2)\rightarrow O(DNM+DM^2)$，NN的输入维度N通常远大于嵌入维度M。做法本质上利用参数共享降低了参数量（trade-off）。

### OPNN
OPNN（向量外积）每次计算得到一个矩阵，也即$\mathbf{P}_{ij}=f_if_j^T\in \R^{M*M}$，作者简单使用了sum pooling降低复杂度

$$
\begin{align}
\mathbf{P}=\Sigma_{i=1}^N\Sigma_{j=1}^N\mathbf{P}_{ij}=\Sigma_{i=1}^N\Sigma_{j=1}^Nf_if_j^T=(\Sigma_{j=1}^Nf_i)(\Sigma_{j=1}^Nf_i)^T\in \R^{M*M}
\end{align}
$$

时空复杂度都由$O(DM^2N^2)\rightarrow O(DMN+DM^2)$，但同时也信息损失严重。

# 一些思考
* 每组特征的embedding和每组特征两两交叉之后的所有内积值 concat到一起 喂给一个mlp 就是PNN。但如果concat完直接喂权重巨大，所以每个节点对应一个权重矩阵，后矩阵分解，通过共享参数降低复杂度（trade-off）。这种思路在例如FM中是相通的。且权重矩阵的思路使用的哈达玛积，在DIN，xDeepFM中都有使用。
* OPNN中，使用“外积+Pooling”交叉特征降维的处理思路，在DIN，xDeepFM中也都有借鉴。
